In [1]:
import requests
import pandas as pd
import json
import re
import glob

from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

In [2]:
data_path = glob.glob("../data/*.json")

In [3]:
data_path

['../data/test.json',
 '../data/train.json',
 '../data/song_meta.json',
 '../data/genre_gn_all.json',
 '../data/val.json']

In [4]:
df_train = pd.read_json("../data/train.json")
df_val = pd.read_json("../data/val.json")
df_test = pd.read_json("../data/test.json")

In [5]:
# tags_list = []
# for i, item in tqdm(df_train.tags.items(), total=115071):
#     tags_list.extend(item)

In [6]:
df_meta = pd.read_json("../data/song_meta.json")

In [7]:
# df_gn = pd.read_json('genre_gn_all.json')

In [8]:
df_meta.describe()

,issue_date,album_id,id
count,7.079890e+05,7.079890e+05,707989.000000
mean,2.003394e+07,3.869081e+06,353994.000000
std,1.263819e+06,3.966919e+06,204378.964204
min,0.000000e+00,1.600000e+01,0.000000
25%,2.008063e+07,4.113540e+05,176997.000000
50%,2.014032e+07,2.276384e+06,353994.000000
75%,2.017023e+07,1.002271e+07,530991.000000
max,2.022011e+07,1.042206e+07,707988.000000


In [9]:
df_train.shape

(115071, 6)

In [10]:
df_val.shape

(23015, 6)

In [11]:
df_test.shape

(10740, 6)

In [12]:
df_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


In [58]:
tag_arr = []
for tags in df_test.tags:
    tag_arr.extend(tags)

In [59]:
tag_set2 = list(set(tag_arr))

In [60]:
tag_set2.sort()

In [56]:
# tag_set

In [62]:
len(tag_set)

29160

In [63]:
len(tag_set2)

2824

In [65]:
tag_same = []
for target in tag_set2:
    left = 0
    right = len(tag_set)
    while right > left:
        mid = (right + left) // 2
        if tag_set[mid] == target:
            tag_same.append(target)
            break
        elif tag_set[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    

In [67]:
len(tag_same)

1293

In [36]:
df_train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [40]:
df_test.plylst_title.nunique()

2040

In [45]:
df_test['song_len'] = df_test.songs.map(lambda x: len(x))

In [47]:
df_test[df_test.song_len == 0]

,tags,id,plylst_title,songs,like_cnt,updt_date,song_len
5,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000,0
7,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000,0
11,"[재즈, 기분전환, 뉴에이지]",135323,책읽으며 듣기좋은 잔잔 BGM♪,[],29,2017-06-13 09:46:27.000,0
17,[],43130,한여름의 센티멘탈 피아노.,[],4,2014-06-04 01:22:05.000,0
21,[새벽],113806,매력있고 편안한 노래들입니다!♬,[],2,2014-12-13 10:23:21.000,0
...,...,...,...,...,...,...,...
10707,[추억],27982,당신이 모르고 지나쳤을 수도 있는 명곡,[],11,2018-05-17 02:53:34.000,0
10709,[사랑],123364,달달하고 꽁냥스러운 염장송 (염장질에 천불주의),[],13,2016-11-08 10:33:32.000,0
10712,[기분전환],61817,피곤함을 달래줄 음악과 함께 하루의 피로를 풀자,[],0,2015-09-03 16:36:35.000,0
10720,[],59912,프랑스를 대표하는 일렉트로니카 아티스트,[],362,2010-01-08 14:53:06.000,0


In [51]:
df_test.song_len.value_counts().sort_index()

0      2043
1         9
2        52
3        73
4       143
       ... 
96        8
97       11
98        8
99       13
100     265
Name: song_len, Length: 101, dtype: int64

In [52]:
df_test

,tags,id,plylst_title,songs,like_cnt,updt_date,song_len
0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000,28
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000,8
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000,40
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000,23
4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000,15
...,...,...,...,...,...,...,...
10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000,34
10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000,37
10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000,15
10738,[여름],2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",4,2015-06-06 09:52:01.000,19


In [49]:
df_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4
...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987


## Data Preprocessing

In [12]:
def data_prep(df, col):
    df = df.explode(col)[["id","songs","like_cnt"]]
    df.columns = ['user_id', 'item_id', "rating"]
    df['rating'] = np.log(df.rating+1).transform(int)
    return df.reset_index(drop=True)

In [13]:
df = pd.concat([df_train, df_val]).reset_index(drop=True)

X_train = data_prep(df, 'songs')
# X_val = data_prep(df_val, 'songs')
# X_test = data_prep(df_test, 'songs')

X_train.rating.value_counts()

In [6]:
df_train

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000
...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000


In [ ]:
# X

# Model

In [16]:
user_enc = LabelEncoder()

In [21]:
result = user_enc.fit_transform(X_train['user_id'])

In [23]:
?csr_matrix

Init signature: csr_matrix(arg1, shape=None, dtype=None, copy=False)
Docstring:     
Compressed Sparse Row matrix

This can be instantiated in several ways:
    csr_matrix(D)
        with a dense matrix or rank-2 ndarray D

    csr_matrix(S)
        with another sparse matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
        is the standard CSR representation where the column indices for
        row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
        corresponding values are stored in ``data[indptr[i]:indptr[i+1]]``.
        If the shape parameter is not supplied, the matrix dimensions
        are inferred 

In [22]:
result

array([55098, 55098, 55098, ..., 58632, 58632, 58632])

In [26]:
class EASE:
    def __init__(self):
        self.user_enc = LabelEncoder()
        self.item_enc = LabelEncoder()

    def _get_users_and_items(self, df):
        users = self.user_enc.fit_transform(df['user_id'])
        print('user done')
        items = self.item_enc.fit_transform(df['item_id'])
        print('items done')
        return users, items

    def fit(self, df, lambda_: float = 0.5, implicit=True):
        """
        df: pandas.DataFrame with columns user_id, item_id and (rating)
        lambda_: l2-regularization term
        implicit: if True, ratings are ignored and taken as 1, else normalized ratings are used
        """
        print("fit Start")
        users, items = self._get_users_and_items(df)
        values = np.ones(df.shape[0]) if implicit else df['rating'].to_numpy() / df['rating'].max()
        
        print("csr_matrix Start")
        X = csr_matrix((values, (users, items)))
        self.X = X
        
        G = X.T.dot(X).toarray()
        diagIndices = np.diag_indices(G.shape[0])
        G[diagIndices] += lambda_
        P = np.linalg.inv(G)
        B = P / (-np.diag(P))
        B[diagIndices] = 0

        self.B = B
        self.pred = X.dot(B)

    def predict(self, train, users, items, k):
        df = pd.DataFrame()
        items = self.item_enc.transform(items)
        dd = train.loc[train.user_id.isin(users)]
        dd['ci'] = self.item_enc.transform(dd.item_id)
        dd['cu'] = self.user_enc.transform(dd.user_id)
        g = dd.groupby('user_id')
        for user, group in tqdm(g):
            watched = set(group['ci'])
            candidates = [item for item in items if item not in watched]
            u = group['cu'].iloc[0]
            pred = np.take(self.pred[u, :], candidates)
            res = np.argpartition(pred, -k)[-k:]
            r = pd.DataFrame({
                "user_id": [user] * len(res),
                "item_id": np.take(candidates, res),
                "score": np.take(pred, res)
            }).sort_values('score', ascending=False)
            df = df.append(r, ignore_index=True)
        df['item_id'] = self.item_enc.inverse_transform(df['item_id'])
        return df

In [27]:
model = EASE()

In [ ]:
model.fit(X_train)

fit Start
user done
items done
csr_matrix Start


In [27]:
X.shape

(5285871, 3)

In [1]:
from scipy.sparse.linalg import inv